It is necessary to install `gdal` for the code below, either using `pip install gdal` (may not build on some systems) or using `apt-get install python3-gdal` and export paths as below.
```bash
export CPLUS_INCLUDE_PATH=/usr/include/gdal
export C_INCLUDE_PATH=/usr/include/gdal
```

In [87]:
from osgeo import gdal
f = gdal.Open('../data/0131.hdf')

In [88]:
f.GetMetadata()

{'AIRSGranuleNumber': '227',
 'AIRSRunTag': '24031200512',
 'ASSOCIATEDINSTRUMENTSHORTNAME.1': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.10': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.11': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.12': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.13': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.14': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.15': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.16': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.17': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.2': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.3': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.4': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.5': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.6': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.7': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.8': 'AIRS',
 'ASSOCIATEDINSTRUMENTSHORTNAME.9': 'AIRS',
 'ASSOCIATEDPLATFORMSHORTNAME.1': 'Aqua',
 'ASSOCIATEDPLATFORMSHORTNAME.10': 'Aqua',
 'ASSOCIATEDPLATFORMSHORTNAME.11': 'Aqua',
 'ASSOCIATEDPLATFORMSHORTNAME.12': 'Aqua',
 'ASSOCIATEDPL

Getting the metadata works. However, our data is not a classic raster data with bands and all. No layers, bands, projection information from `gdal`.

In [89]:
print("'ds' type", type(f))
print("Projection: ", f.GetProjection())  # get projection
print("Columns:", f.RasterXSize)  # number of columns
print("Rows:", f.RasterYSize)  # number of rows
print("Band count:", f.RasterCount)  # number of bands
print("GeoTransform", f.GetGeoTransform())

'ds' type <class 'osgeo.gdal.Dataset'>
Projection:  
Columns: 512
Rows: 512
Band count: 0
GeoTransform (0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [90]:
print("Driver: {}/{}".format(f.GetDriver().ShortName,
                            f.GetDriver().LongName))
print("Size is {} x {} x {}".format(f.RasterXSize,
                                    f.RasterYSize,
                                    f.RasterCount))
print("Projection is {}".format(f.GetProjection()))
geotransform = f.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: HDF4/Hierarchical Data Format Release 4
Size is 512 x 512 x 0
Projection is 
Origin = (0.0, 0.0)
Pixel Size = (1.0, 1.0)


In [91]:
data_array = f.GetRasterBand(1).ReadAsArray()

AttributeError: 'NoneType' object has no attribute 'ReadAsArray'

Among the subdatasets, we could find radiances, but not geolocation information including latitude and longitude, nor time. All of the three should be 2D arrays with size 135x90.

In [92]:
f.GetSubDatasets()

[('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:radiances',
  '[135x90x2645] radiances L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:scanang',
  '[135x90] scanang L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:satzen',
  '[135x90] satzen L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:satazi',
  '[135x90] satazi L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:solzen',
  '[135x90] solzen L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:solazi',
  '[135x90] solazi L1C_AIRS_Science (32-bit floating-point)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:sun_glint_distance',
  '[135x90] sun_glint_distance L1C_AIRS_Science (16-bit integer)'),
 ('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:to

In [93]:
radiances = gdal.Open('HDF4_EOS:EOS_SWATH:"../data/0131.hdf":L1C_AIRS_Science:radiances')
rad = radiances.ReadAsArray()
(rad.min(), rad.max(), rad.mean(), rad.std())
rad

array([[[4.87381363e+01, 4.97418594e+01, 4.92623367e+01, ...,
         2.91861951e-01, 2.93189764e-01, 2.89436787e-01],
        [4.84533043e+01, 4.92636070e+01, 5.02469215e+01, ...,
         2.17256114e-01, 2.20819712e-01, 2.11136296e-01],
        [4.87467461e+01, 4.94835815e+01, 4.90431747e+01, ...,
         2.17728987e-01, 2.17840418e-01, 2.16097727e-01],
        ...,
        [4.55192146e+01, 4.74843102e+01, 4.57535591e+01, ...,
         8.98296088e-02, 8.73484910e-02, 8.69715959e-02],
        [4.55038376e+01, 4.64851379e+01, 4.55226402e+01, ...,
         1.03990264e-01, 1.01502538e-01, 1.00108139e-01],
        [4.59744606e+01, 4.64996567e+01, 4.65012321e+01, ...,
         1.58180982e-01, 1.58249259e-01, 1.59164071e-01]],

       [[4.92013626e+01, 4.84931679e+01, 4.85151482e+01, ...,
         3.58707249e-01, 3.56596053e-01, 3.50922883e-01],
        [4.74814606e+01, 4.90044556e+01, 4.92540970e+01, ...,
         2.23079637e-01, 2.25261018e-01, 2.22216144e-01],
        [4.89828377e+01, 

The code above read radiances as an array, and the code below write this array as parquet.

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

pa_table = pa.table({"data": rad[0, 0]})
pq.write_table(pa_table, "../data/0131.01.parquet")